# Power-law distributed data

This notebook demonstrates how the sample mean of power-law distributed data does not provide useful estimates of the population mean.

In [1]:
import altair as alt
import numpy as np
import pandas as pd
from math import log

from scipy.stats import poisson, zipf

In [2]:
# Parameters

mn = 15

# poisson
mu = mn

# zeta (zipf) with a mean of ~ 15
a = 2.04251395975

# graph dimensions for our plots
width=600
height=400

In [3]:
# Verify that means are 15
[poisson.stats(mu, moments='m'),
 zipf.stats(a, moments='m')
]

[np.float64(15.0), np.float64(15.000000015276777)]

# Distributions

In [57]:
# pmfs for the two distributions
xmax = 480
x = np.arange(1, xmax)
df_pf = pd.DataFrame({
    'x': x,
    'poisson': poisson.pmf(x, mu),
    'zeta': zipf.pmf(x, a)
})

In [59]:
xmin = 0
xmax = 40
ymin = 0
ymax = 0.65

alt.Chart(df_pf).mark_bar().encode(
    x=alt.X(
        'x', 
        title="Incident duration (min)", 
        scale=alt.Scale(domain=(xmin, xmax))
    ),
    y=alt.Y(
        'poisson', 
        title='Probability', 
        scale=alt.Scale(domain=(ymin,ymax))
    )
).transform_filter(
    alt.FieldRangePredicate(
        field='x',
        range = [xmin, xmax]
    )
).properties(
    title='Poisson pmf (μ=15)',
    width=width,
    height=height
)

alt.Chart(...)

In [60]:
xmin = 0
xmax = 40
ymin = 0
ymax = 0.65


alt.Chart(df_pf).mark_bar().encode(
    x=alt.X('x', 
            title="Incident duration (min)", 
            scale=alt.Scale(domain=(xmin, xmax))),
    y=alt.Y('zeta', title='Probability')
).transform_filter(
    alt.FieldRangePredicate(
        field='x',
        range = [xmin, xmax]
    )
).properties(
    title='Zeta pmf (s=2.04251395975)',
    width=width,
    height=height
)

alt.Chart(...)

In [73]:
# Centered around 120 min

xmin = 100
xmax = 140
#ymin = 0
#ymax = 5.5e-5
ymin=1e-10
ymax=1e-4


alt.Chart(df_pf).mark_bar().encode(
    x=alt.X('x', 
            title="Incident duration (min)", 
            scale=alt.Scale(domain=(xmin, xmax))),
    y=alt.Y('zeta', 
            title='Probability',
            axis=alt.Axis(format=".1e")
           )
).transform_filter(
    alt.FieldRangePredicate(
        field='x',
        range = [xmin, xmax]
    )
).properties(
    title='Zeta pmf (s=2.04251395975)',
    width=width,
    height=height
)

alt.Chart(...)

In [74]:
xmin = 100
xmax = 140
ymin = 0
ymax = 5.5e-5

alt.Chart(df_pf).mark_bar().encode(
    x=alt.X(
        'x', 
        title="Incident duration (min)", 
        scale=alt.Scale(domain=(xmin, xmax))
    ),
    y=alt.Y(
        'poisson', 
        title='Probability',
        axis=alt.Axis(format=".1e")
    )
).transform_filter(
    alt.FieldRangePredicate(
        field='x',
        range = [xmin, xmax]
    )
).properties(
    title='Poisson pmf (μ=15)',
    width=width,
    height=height
)

alt.Chart(...)

# Random samples

In [76]:
# Generate N random data points for each distribution
N = 5000
rvs_poisson = poisson.rvs(mu=mu, size=N)
rvs_zeta = zipf.rvs(a, size=N)
df_rvs = pd.DataFrame({
    'poisson': rvs_poisson,
    'zeta': rvs_zeta,
    'n' : np.arange(1, N+1)
})

In [77]:
# Plot both random variables on a scatter plot

alt.Chart(df_rvs).mark_point().encode(
    x=alt.X('n'),
    y=alt.Y('poisson', title='incident duration (min)')
).properties(title='Poisson random samples', width=600, height=400).display()

alt.Chart(df_rvs).mark_point().encode(
    x=alt.X('n'),
    y=alt.Y('zeta', title='incident duration (min)')
).properties(title='Zeta random samples', width=600, height=400).display()

alt.Chart(...)

alt.Chart(...)

In [84]:
# Plot both random variables on a scatter plot
ymin=1
ymax=2000

alt.Chart(df_rvs).mark_point().encode(
    x=alt.X('n'),
    y=alt.Y('poisson', title='incident duration (min)', scale=alt.Scale(domain=(ymin,ymax),type="log"))
).properties(title='Poisson random samples', width=600, height=400).display()

alt.Chart(df_rvs).mark_point().encode(
    x=alt.X('n'),
    y=alt.Y('zeta', title='incident duration (min)', scale=alt.Scale(domain=(ymin,ymax),type="log"))
).properties(title='Zeta random samples', width=600, height=400).display()

alt.Chart(...)

alt.Chart(...)

# Thought experiment

Look at averages of random samples

In [85]:
def tumbling_window_average(arr, window_size=25):
    "This is a tumbling window as opposed to a sliding window"
    windows = len(arr) // window_size
    return arr.reshape(windows, window_size).mean(axis=1)

In [88]:
wsize = 25
Nw = N // wsize
t = np.arange(1, Nw+1)

df_win = pd.DataFrame({
    'poisson': tumbling_window_average(rvs_poisson),
    'zeta': tumbling_window_average(rvs_zeta),
    'mean': np.repeat(mn,Nw),
    'time': t
})

In [95]:
alt.Chart(df_win).mark_point(opacity=0.7).encode(x='time', y='poisson') + \
alt.Chart(df_win).mark_line().encode(
    x=alt.X('time'),
    y=alt.Y('poisson',title="average incident duration (min)", scale=alt.Scale(domain=(0,70)))
).properties(title='Poisson', width=600, height=400) + \
alt.Chart(df_win).mark_rule(color='red', strokeDash=[5, 5]).encode(y='mean')

alt.LayerChart(...)

In [97]:
alt.Chart(df_win).mark_point(opacity=0.7).encode(x='time', y='zeta') + \
alt.Chart(df_win).mark_line().encode(
    x=alt.X('time'),
    y=alt.Y('zeta',title="average incident duration (min)", scale=alt.Scale(domain=(0,70)))
).properties(title='Zeta', width=600, height=400) + \
alt.Chart(df_win).mark_rule(color='red', strokeDash=[5, 5]).encode(y='mean')

alt.LayerChart(...)

In [ ]:
alt.Chart(df_win).mark_point(opacity=0.7).encode(x='time', y='zeta') + \
alt.Chart(df_win).mark_line().encode(
    x=alt.X('time'),
    y=alt.Y('zeta',title="average incident duration (min)", scale=alt.Scale(domain=(0,70)))
).properties(title='Zeta', width=600, height=400) + \
alt.Chart(df_win).mark_rule(color='red', strokeDash=[5, 5]).encode(y='mean')

In [105]:


alt.Chart(df_win).mark_point(opacity=0.7).encode(x='time', y='zeta').transform_filter(alt.datum.time <= 12) + \
alt.Chart(df_win).mark_line().encode(
    x=alt.X('time', scale=alt.Scale(domain=(1,12)), title="time (months)"),
    y=alt.Y('zeta',title="average incident duration (min)", scale=alt.Scale(domain=(0,70)))
).transform_filter(alt.datum.time <= 12
).properties(title='Zeta', width=1200, height=800)

alt.LayerChart(...)